IMPORTS

In [8]:
import whisper 
import speech_recognition as sr
import torch
import numpy as np
import wave

CHECKING FOR MY GPU

In [12]:
device = "cuda" if torch.cuda.is_available() else "cpu"
if device == "cuda":
    print("Using GPU")
else:
    print("Using CPU")

Using GPU


IMPORTING LANGAUGE MODEL

In [13]:
model = whisper.load_model("turbo", device=device)

INITIALIZE MIC & RECOGNIZER

In [14]:
mic = sr.Microphone(sample_rate=16000)
recognizer = sr.Recognizer()

LOGIC FOR AUDIO RECOGNITION

In [21]:
with mic as source:

    print("Please start speaking")
    recognizer.adjust_for_ambient_noise(source, duration=1.5)
    audio = recognizer.listen(source, phrase_time_limit=7)

    with open("audio.wav", "wb") as f:
        f.write(audio.get_wav_data())
    print("Audio file saved")

    audio_data = np.frombuffer(audio.get_wav_data(), dtype=np.int16).flatten().astype(np.float32) / 32768
    result = model.transcribe(audio_data)

    print(result["text"])

Please start speaking
Audio file saved
 pick the red box and keep it on the floor
